In [1]:
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf 

class SlidingLayer(Layer):

    def __init__(self, window_size, **kwargs):
        self.window_size = window_size
        super().__init__(**kwargs)

    def build(self, input_shape):
        super().build(input_shape)

    def call(self, batch):
        s = batch.get_shape().as_list();
        # if (not s[0] == None) and (not s[0] == 1):
        # raise BaseException(s)
        outputs = []
        w = self.window_size
        
        for i in range(s[1]):
            for j in range(s[2]):                    
                if w + i > s[1]-1 or w + i > s[2]-1:
                    continue
                    
                outputs.append(tf.slice(batch[0], [i, j, 0], [w, w, -1]))

        return K.stack(outputs)
        
    
    def compute_output_shape(self, input_shape):
        c = 0
        w = self.window_size
        for i in range(input_shape[1]):
            for j in range(input_shape[2]):                    
                if w + i > input_shape[1]-1 or w + i > input_shape[2]-1:
                    continue
                c += 1
        
        print(input_shape)
        print(c)
        
        return (c, self.window_size, self.window_size, input_shape[3])

Using TensorFlow backend.


In [23]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [24]:
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf

class RoIExtractorLayer(Layer):
    
    def __init__(self, n_rois, window, **kwargs):
        
        assert len(window) == 2, 'window is a tuple HxW' 
       
        self.n_rois = n_rois
        self.window = window

        super().__init__(**kwargs)
    
    def build(self, input_shape):
        super().build(input_shape)
    
    #def max_pool(self, tensor, size_h, size_w):
    #    (h, w, d) = tuple(tensor.get_shape().as_list());
        
        # grid size
    #    h_H = K.cast(h/float(size_h), 'int32') 
    #    w_W = K.cast(w/float(size_w), 'int32') 

    #    outputs = []
        
    #    for i in range(size_h):
    #        for j in range(size_w):
    #            y1 = K.cast(i * h_H, 'int32') 
    #            x1 = K.cast(j * w_W, 'int32')

    #            pool_volume = tf.slice(tensor, [y1, x1, 0], [h_H, w_W, -1])
    #            pooled_val = K.max(pool_volume, axis=(0, 1))              
    #            outputs.append(pooled_val)

    #    final_output = K.stack(outputs)
    #   final_output = K.reshape(final_output, (size_h, size_w, d))
    #    
    #    return final_output
                
    
    def call(self, x):
        
        assert len(x) == 2, 'RoILayer requires both features and regions' 
        
        feature_map = x[0]
        # cls = x[1]
        reg = x[1]

        feature_map_s = feature_map.get_shape().as_list();

        
        reg_s = reg.get_shape().as_list();
        H = self.window[0]
        W = self.window[1]

        # print(reg_s)

        
        # assert reg_s[3]  % 4 == 0, 'Rois format should be batch_sizex4xn'
        
        outputs = []
        
        
        for i in range(reg_s[1]):
            # for j in range(int(reg_s[3]/4)):
                
                # print(tf.is_nan(reg[i, 0, 0, j+0]))
                
                # print(reg[i, 0, 0, j+0])
    
                
                # print(reg_s)
            
            y = K.cast(reg[0, i, 0],'int32') 
            x = K.cast(reg[0, i, 1],'int32')
            h = K.cast(reg[0, i, 2],'int32') 
            w = K.cast(reg[0, i, 3],'int32') 

            rs = tf.image.resize_images(feature_map[:, y:y+h, x:x+w, :], (H,W))
            outputs.append(rs)
                
                # print((j*4))
                
                # print(r)
                # print(c)
                # print(h)
                # print(w)
                
                #r = K.minimum(r,0)
                #c = K.maximum(h,0)
                
                # slc = tf.slice(feature_map, [0, r, c, 0], [-1, h, w, -1])
                
                # outputs.append(self.max_pool(feature_map[0],H,W))
                
        final_output = K.concatenate(outputs, axis=0)
        final_output = K.reshape(final_output, (1, reg_s[1], H, W, feature_map_s[3]))
        print(final_output)
        
        return final_output
    
    def compute_output_shape(self, input_shape):
        print(input_shape)
        return (1, None, self.window[0], self.window[1] ,input_shape[0][3])

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Reshape, Flatten, TimeDistributed

input_layer = Input(shape=(32,32,3))

# general conv layers
conv_layer = Conv2D(16, (3,3), padding='same')(input_layer)

# rpn 
n_ratios = 2
n_scales = 2
n_boxes = n_ratios * n_scales

# sliding_layer = SlidingLayer(3)(conv_layer)
intermediate_layer = Conv2D(256, (3,3))(conv_layer)
cls_layer = Conv2D(2 * n_boxes, (1,1))(intermediate_layer)
reg_layer = Conv2D(4 * n_boxes, (1,1))(intermediate_layer)

reshape_layer = Reshape((30*30*4,4))(reg_layer)


# roi layer
roi_layer = RoIExtractorLayer(928, (16,16), batch_size=1)([conv_layer, reshape_layer])


# roi specific conv layers
roi_conv_layer_1 = TimeDistributed(Conv2D(256, (3,3), padding='same'))(roi_layer)
roi_conv_layer_2 = TimeDistributed(Conv2D(256, (3,3), padding='same'))(roi_conv_layer_1)


# classification + location regresssion
# n_classes = 16
# roi_cls_layer = TimeDistributed(roi_conv_layer_2)(Conv2D(n_classes, (16,16)))
# roi_reg_layer = TimeDistributed(roi_conv_layer_2)(Conv2D(4, (16,16)))




model = Model(
    inputs = input_layer, 
    #outputs = [sliding_layer],
    outputs = [cls_layer, reg_layer, roi_layer],
    #loss = ['categorical_crossentropy', 'mean_squared_error', 'categorical_crossentropy', 'mean_squared_error']
)

In [7]:
model.compile(loss='mean_absolute_error', optimizer='adadelta')
res = model.predict_on_batch(x_train[0:1])
print(res.shape)
#model.fit(x=x_train, y=y_train, epochs=10, batch_size=1)

NameError: name 'model' is not defined